This repo contains the code base of the project that could be run on Google Colab.

**Code structure :**
1. Upload the data
2. Set up the dataset
3. Define the machine learning model
4. Perform 10-fold validation
5. Calculate Intergrated Gradient


**1. Upload the data**

1.1 Unzip and Upload the [microstructure-property datasets](http://cs.wisc.edu/~demirel/data.tar.gz) to the google drive.

1.2 Mount the google drive folder and go to the folder contains the subfolder data/


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My\ Drive/microstructure/GNN

**2. Set up the dataset**

There are 492 microstructures in total. Four or five different magnetic fields are applied to each microstructure, amounting to 2287 data points.

- neighbor.txt: adajcency matrix
- feature.txt: feature matrix
- property.txt: external magnetic field and corresponding magnetostriction

2.1 data reading and preprocessing


In [2]:
from __future__ import print_function
import numpy as np
import torch
from torch.utils.data import Dataset
from scipy import sparse

class GraphDataSet(Dataset):
    def __init__(self, num_graphs, max_node, num_features):
        for i in range(1, num_graphs + 1):
            # load files
            file_paths = ['data/structure-{}/neighbor.txt'.format(i), 'data/structure-{}/feature.txt'.format(i),
                          'data/structure-{}/property.txt'.format(i)]

            graph_elements = [np.loadtxt(file_paths[0]), np.loadtxt(file_paths[1]), np.loadtxt(file_paths[2])]

            # feature data manipulation
            graph_elements[1] = manipulate_feature(graph_elements[1], max_node, num_features)

            # normalize the adjacency matrix
            graph_elements[0] = normalize_adj(graph_elements[0], max_node)

            # delete data points with negative properties
            graph_elements[2] = graph_elements[2][graph_elements[2].min(axis=1) >= 0, :]
            # get the dimension of proprty
            num_properties, width = np.shape(graph_elements[2])
            # independent variable t, the external field
            t = np.delete(graph_elements[2], 1, axis=1)
            # label, the magnetostriction
            label = np.delete(graph_elements[2], 0, axis=1)

            # change it to the several data points
            multiple_neighbor, multiple_feature = [graph_elements[0] for x in range(num_properties)], \
                                                  [graph_elements[1] for x in range(num_properties)]

                # concatenating the matrices
            if i == 1:
                adjacency_matrix, node_attr_matrix, t_matrix, label_matrix = multiple_neighbor, multiple_feature, t, label
            else:
                adjacency_matrix, node_attr_matrix, t_matrix, label_matrix = np.concatenate((adjacency_matrix, multiple_neighbor)), \
                                                                             np.concatenate((node_attr_matrix, multiple_feature)), \
                                                                             np.concatenate((t_matrix, t)),\
                                                                             np.concatenate((label_matrix, label))

        # normalize the independent variable t matrix
        t_matrix, label_matrix = normalize_t_label(t_matrix, label_matrix)

        self.adjacency_matrix = np.array(adjacency_matrix)
        self.node_attr_matrix = np.array(node_attr_matrix)
        self.t_matrix = np.array(t_matrix)
        self.label_matrix = np.array(label_matrix)

        print('--------------------')
        print('Training Data:')
        print('adjacency matrix:\t', self.adjacency_matrix.shape)
        print('node attribute matrix:\t', self.node_attr_matrix.shape)
        print('t matrix:\t\t', self.t_matrix.shape)
        print('label name:\t\t', self.label_matrix.shape)
        print('--------------------')

    def __len__(self):
        return len(self.adjacency_matrix)

    def __getitem__(self, idx):
        adjacency_matrix = self.adjacency_matrix[idx].todense()
        node_attr_matrix = self.node_attr_matrix[idx].todense()
        t_matrix = self.t_matrix[idx]
        label_matrix = self.label_matrix[idx]

        adjacency_matrix = torch.from_numpy(adjacency_matrix)
        node_attr_matrix = torch.from_numpy(node_attr_matrix)
        t_matrix = torch.from_numpy(t_matrix)
        label_matrix = torch.from_numpy(label_matrix)
        return adjacency_matrix, node_attr_matrix, t_matrix, label_matrix

def normalize_adj(neighbor, max_node):
    np.fill_diagonal(neighbor, 1)  # add the identity matrix
    D = np.sum(neighbor, axis=0)  # calculate the diagnoal element of D
    D_inv = np.diag(np.power(D, -0.5))  # construct D
    neighbor = np.matmul(D_inv, np.matmul(neighbor, D_inv))  # symmetric normalization of adjacency matrix

    # match dimension to the max dimension for neighbors
    result = np.zeros((max_node, max_node))
    result[:neighbor.shape[0], :neighbor.shape[1]] = neighbor
    neighbor = result

    # convert the feature matrix to sparse matrix
    neighbor = sparse.csr_matrix(neighbor)

    return neighbor

def manipulate_feature(feature, max_node, features):
    feature = np.delete(feature, 0, axis=1)  # remove the first column (Grain ID)
    feature[:, [3]] = (feature[:, [3]] - np.mean(feature[:, [3]])) / np.std(
        feature[:, [3]])  # normalize grain size
    feature[:, [4]] = (feature[:, [4]] - np.mean(feature[:, [4]])) / np.std(
        feature[:, [4]])  # normalize number of neighbors

    # match dimension to the max dimension for features
    result = np.zeros((max_node, features))
    result[:feature.shape[0], :feature.shape[1]] = feature
    feature = result

    # convert the feature matrix to sparse matrix
    feature = sparse.csr_matrix(feature)

    return feature

def normalize_t_label(t_matrix, label_matrix):
    t_matrix = t_matrix / 10000
    label_mean = np.mean(label_matrix)
    label_std = np.std(label_matrix)
    label_matrix = (label_matrix - label_mean) / label_std

    # save the mean and standard deviation of label
    norm = np.array([label_mean, label_std])
    np.savez_compressed('norm.npz', norm=norm)

    return t_matrix, label_matrix

2.2 dataloader for training set and testing set. 

The training set and testing set are loaded according to the input indices.

In [4]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

def get_data(idx_path, running_index, folds, batch_size, max_node, num_features, num_graphs):
    indices = np.load(idx_path, allow_pickle=True)['indices']
    test_idx = indices[running_index]
    train_idx = indices[[i for i in range(folds) if i != running_index]]
    train_idx = [item for sublist in train_idx for item in sublist]

    dataset = GraphDataSet(num_graphs, max_node, num_features)
    train_data = DataLoader(dataset, batch_size=batch_size, sampler=SubsetRandomSampler(train_idx))
    test_data = DataLoader(dataset, batch_size=batch_size, sampler=SubsetRandomSampler(test_idx))
    return train_data, test_data

**3. Define the machine learning models**

3.1 define some functions to
- convert tensor to variable
- convert variable to numpy array
- output the true value and predicted value of the dataset
- calculate the mean square error (MSE)
- calculate the macro average relative error (MARE)



In [10]:
from torch.autograd import Variable

def tensor_to_variable(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x.float())

def variable_to_numpy(x):
    if torch.cuda.is_available():
        x = x.cpu()
    x = x.data.numpy()
    return x

def print_preds(y_label_list, y_pred_list, test_or_tr, running_index):
    length, w = np.shape(y_label_list)
    filename="{}_Output_{}.txt".format(test_or_tr, running_index)
    output=open(filename, "w")
    print('{} Set Predictions: '.format(test_or_tr), file = output, flush = True)
    print('True_Value Predicted_value', file = output, flush = True)
    for i in range(0, length):
        print('%f, %f' % (y_label_list[i], y_pred_list[i]), file = output, flush = True)

def mse(Y_prime, Y):
    return np.mean((Y_prime - Y) ** 2)

def macro_avg_err(Y_prime, Y):
    if type(Y_prime) is np.ndarray:
        return np.sum(np.abs(Y - Y_prime)) / np.sum(np.abs(Y))
    return torch.sum(torch.abs(Y - Y_prime)) / torch.sum(torch.abs(Y))

3.2 define the Graph Neural Network model

In [6]:
import argparse
import time
from collections import OrderedDict
import os
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# the MPL layer
class Message_Passing(nn.Module):
    def forward(self, x, adjacency_matrix):
        neighbor_nodes = torch.bmm(adjacency_matrix, x)
        logging.debug('neighbor message\t', neighbor_nodes.size())
        logging.debug('x shape\t', x.size())
        return x

# the GNN model
class GraphModel(nn.Module):
    def __init__(self, max_node_num, atom_attr_dim, latent_dim):
        super(GraphModel, self).__init__()

        self.max_node_num = max_node_num    # max number of grains/nodes for microstructures in the dataset
        self.atom_attr_dim = atom_attr_dim  # number of features of each grain/node before passing MPLs
        self.latent_dim = latent_dim        # number of features of each grain/node after passing MPls

        # MPLs
        self.graph_modules = nn.Sequential(OrderedDict([
            ('message_passing_0', Message_Passing()),
            ('dense_0', nn.Linear(self.atom_attr_dim, 50)),
            ('activation_0', nn.Sigmoid()),
            ('message_passing_1', Message_Passing()),
            ('dense_1', nn.Linear(50, self.latent_dim)),
            ('activation_1', nn.Sigmoid()),
        ]))
        
        # FLs
        self.fully_connected = nn.Sequential(
            nn.Linear(self.max_node_num * self.latent_dim + 1, 1024),
            nn.ReLU(),
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

        return

    def forward(self, node_attr_matrix, adjacency_matrix, t_matrix):
        node_attr_matrix = node_attr_matrix.float()
        adjacency_matrix = adjacency_matrix.float()
        x = node_attr_matrix
        logging.debug('shape\t', x.size())

        for (name, module) in self.graph_modules.named_children():
            if 'message_passing' in name:
                x = module(x, adjacency_matrix=adjacency_matrix)
            else:
                x = module(x)

        # Before flatten, the size should be [Batch size, max_node_num, latent_dim]
        logging.debug('size of x after GNN\t', x.size())
        # After flatten is the graph representation
        x = x.view(x.size()[0], -1)
        logging.debug('size of x after GNN\t', x.size())

        # Concatenate [x, t]
        x = torch.cat((x, t_matrix), 1)

        x = self.fully_connected(x)
        return x

3.3 define functions to
- perform training
- perform testing

In [7]:
def train(model, train_dataloader, test_dataloader, optimizer, criterion, epochs, running_index):

    print()
    print("*** Training started! ***")
    print()

    filename="Learning_Output_{}.txt".format(running_index)
    output=open(filename, "w")
    print('Epoch Training_time Training_MSE Testing_MSE',file = output, flush = True)

    for epoch in range(epochs):
        model.train()

        train_start_time = time.time()

        for batch_id, (adjacency_matrix, node_attr_matrix, t_matrix, label_matrix) in enumerate(train_dataloader):
            adjacency_matrix = tensor_to_variable(adjacency_matrix)
            node_attr_matrix = tensor_to_variable(node_attr_matrix)
            t_matrix = tensor_to_variable(t_matrix)
            label_matrix = tensor_to_variable(label_matrix)

            optimizer.zero_grad()

            y_pred = model(adjacency_matrix=adjacency_matrix, node_attr_matrix=node_attr_matrix, t_matrix=t_matrix) # model prediction
            loss = criterion(y_pred, label_matrix) # calculate loss
            loss.backward()    # back propagation
            optimizer.step()   # update weight

        train_end_time = time.time()
        _, training_MSE = test(model, train_dataloader, 'Training', False, criterion,running_index)
        _, testing_MSE  = test(model, test_dataloader,  'Test',     False, criterion,running_index)
        print('%d %.3f %e %e' % (epoch, train_end_time-train_start_time, training_MSE, testing_MSE), file = output, flush = True)
    
    output.close()

def test(model, data_loader, test_or_tr, printcond, criterion, running_index):
    model.eval()
    if data_loader is None:
        return None, None

    y_label_list, y_pred_list, total_loss = [], [], 0

    for batch_id, (adjacency_matrix, node_attr_matrix, t_matrix, label_matrix) in enumerate(data_loader):
        adjacency_matrix = tensor_to_variable(adjacency_matrix)
        node_attr_matrix = tensor_to_variable(node_attr_matrix)
        t_matrix = tensor_to_variable(t_matrix)
        label_matrix = tensor_to_variable(label_matrix)

        y_pred = model(adjacency_matrix=adjacency_matrix, node_attr_matrix=node_attr_matrix, t_matrix=t_matrix)

        y_label_list.extend(variable_to_numpy(label_matrix))
        y_pred_list.extend(variable_to_numpy(y_pred))

    norm = np.load('norm.npz', allow_pickle=True)['norm']
    label_mean, label_std = norm[0], norm[1]

    # get the original value of true value and predicted value
    y_label_list = np.array(y_label_list) * label_std + label_mean
    y_pred_list = np.array(y_pred_list) * label_std + label_mean

    # calculate the MARE and MSE 
    total_MARE = macro_avg_err(y_pred_list, y_label_list)
    total_mse = criterion(torch.from_numpy(y_pred_list), torch.from_numpy(y_label_list)).item()

    if printcond:
        print_preds(y_label_list, y_pred_list, test_or_tr, running_index)
        

    return total_MARE, total_mse



**4. Perform 10-fold validation**

4.1 split the whole dataset into ten folds

In [ ]:
import argparse
from sklearn.model_selection import KFold

# split the indices into different folds
def split_data(num_folds, num_graphs, max_node, num_features):
    dataset = GraphDataSet(num_graphs, max_node, num_features)
    num_of_data = dataset.__len__()
    kf = KFold(n_splits=num_folds, shuffle=True)
    ind = []
    for i, (_, index) in enumerate(kf.split(np.arange(num_of_data))):
        np.random.shuffle(index)
        ind.append(index)
    ind = np.array(ind, dtype=object)
    return ind

# save the indices
def extract_graph_data(out_file_path, ind):
    np.savez_compressed(out_file_path, indices = ind)

num_folds = 10                  # number of folds
out_file_path='indices.npz'   # output file path
split_seed = 123              # random seed
np.random.seed(split_seed)    # set the random seed

num_graphs = 492
max_node = 300
num_features = 5

print("Output File Path: {}".format(out_file_path))

indices = split_data(num_folds,num_graphs, max_node, num_features)
extract_graph_data(out_file_path, indices)

print("Data successfully split into {} folds!".format(num_folds))

4.2 perform 10-fold validation

In [ ]:
num_graph = 492
max_node_num = 300 
atom_attr_dim = 5

# hyperparameters
latent_dim = 5
epochs = 1000
batch_size = 32
learning_rate = 1e-4
min_learning_rate = 1e-5
train_seed = 123
folds = 10
idx_path = 'indices.npz'

# set random seed
np.random.seed(train_seed)
torch.manual_seed(train_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(train_seed)
    torch.cuda.manual_seed(train_seed)
torch.backends.cudnn.deterministic = True

for i in range(folds):
    running_index = i

    # Define the model
    model = GraphModel(max_node_num=max_node_num, atom_attr_dim=atom_attr_dim, latent_dim=latent_dim)
    if torch.cuda.is_available():
        model.cuda()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) # optimizer
    criterion = nn.MSELoss()  # loss

    # get the data
    train_dataloader, test_dataloader = get_data(idx_path, 
                                                 running_index, 
                                                 folds, 
                                                 batch_size, 
                                                 max_node, 
                                                 num_features, 
                                                 num_graphs)

    # train the mode
    train(model, train_dataloader, test_dataloader, optimizer, criterion, epochs, running_index)

    # predictions on the entire training and test datasets
    train_rel, train_mse = test(model, train_dataloader, 'Training', True, criterion, running_index)
    test_rel, test_mse = test(model, test_dataloader, 'Testing', True, criterion, running_index)

    print()
    print('--------------------')
    print()
    print("Running index: {}".format(running_index))
    print("Training Relative Error: {:.3f}%".format(100 * train_rel))
    print("Test Relative Error: {:.3f}%".format(100 * test_rel))
    print("Training MSE: {}".format(train_mse))
    print("Test MSE: {}".format(test_mse))


**5. Calculate Intergrated Gradient**

5.1 define functions to 
- calculate the gradient


In [12]:
import math
def tensor_to_variable_grad(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x.float(),requires_grad=True)

def gradient_calculation(adjacency_matrix, node_attr_matrix, t_matrix, model):
    adjacency_matrix=tensor_to_variable_grad(adjacency_matrix)
    node_attr_matrix=tensor_to_variable_grad(node_attr_matrix)
    t_matrix=tensor_to_variable_grad(t_matrix)
    
    label=model(adjacency_matrix=adjacency_matrix, node_attr_matrix=node_attr_matrix, t_matrix=t_matrix)
    label.backward()  # back propagation
    grad_node_attr_matrix=variable_to_numpy(node_attr_matrix.grad) # get the gradient of features
    grad_t_matrix=variable_to_numpy(t_matrix.grad)  # get the gradient of external field
    
    return grad_node_attr_matrix, grad_t_matrix


- get a real input and define the corresponding baseline input

In [91]:
def readinput(id, max_node, num_features):
    # load files
    file_paths = ['data/structure-{}/neighbor.txt'.format(id), 'data/structure-{}/feature.txt'.format(id),
                  'data/structure-{}/property.txt'.format(id)]
    
    graph_elements = [np.loadtxt(file_paths[0]), np.loadtxt(file_paths[1]), np.loadtxt(file_paths[2])]
    
    # feature data manipulation
    graph_elements[1] = manipulate_feature(graph_elements[1], max_node, num_features)
    
    # normalize the adjacency matrix
    graph_elements[0] = normalize_adj(graph_elements[0], max_node)

    # delete data points with negative properties
    graph_elements[2] = graph_elements[2][graph_elements[2].min(axis=1) >= 0, :]

    # get the dimension of property
    num_properties, width = np.shape(graph_elements[2])
    proprty = graph_elements[2]

    # get the field of the last data point asoociated with this graph.
    t_matrix = proprty[num_properties-1,0].reshape((1,1))
    t_matrix = t_matrix / 10000

    adjacency_matrix = torch.from_numpy(graph_elements[0].todense())
    node_attr_matrix = torch.from_numpy(graph_elements[1].todense())
    t_matrix = torch.from_numpy(t_matrix)

    return adjacency_matrix, node_attr_matrix, t_matrix

def baselineinput(node_attr_matrix):
    # specify the baseline: same with input graph except the Euler angles
    baseline_node_attr_matrix = torch.zeros(node_attr_matrix.size())
    
    alpha, beta, gamma = 0.5*math.pi, 0.5*math.pi, 0.5*math.pi

    for i in range(node_attr_matrix.size()[0]):
        if node_attr_matrix[i][0] !=0 or node_attr_matrix[i][1] !=0 or node_attr_matrix[i][2]!=0:
            baseline_node_attr_matrix[i][0]=alpha
            baseline_node_attr_matrix[i][1]=beta
            baseline_node_attr_matrix[i][2]=gamma
        
        baseline_node_attr_matrix[i][3]=node_attr_matrix[i][3]
        baseline_node_attr_matrix[i][4]=node_attr_matrix[i][4]

    return baseline_node_attr_matrix


- calculate the intergrated gradient

In [86]:
def Intergrated_gradient_calculation(id, max_node, num_features, model,steps=200):
    # read real input
    adjacency_matrix, node_attr_matrix, t_matrix = readinput(id, max_node, num_features)

    # get baseline input
    baseline_node_attr_matrix = baselineinput(node_attr_matrix)

    # define the gradient matrix
    grad_node_attr_matrix=torch.zeros(1,max_node,num_features)

    # change the dimension of adjacency matrix and t matrix      
    adjacency_matrix = torch.reshape(adjacency_matrix,(1,max_node, max_node))
    
    for step in range(steps):
        # define temporary feature matrix
        temp_node_attr_matrix = baseline_node_attr_matrix + (node_attr_matrix-baseline_node_attr_matrix) * step / steps
        temp_node_attr_matrix = torch.reshape(temp_node_attr_matrix,(1,max_node,num_features))
        
        temp_grad_node_attr_matrix, _ = gradient_calculation(adjacency_matrix, temp_node_attr_matrix, t_matrix, model)
        grad_node_attr_matrix += temp_grad_node_attr_matrix

    grad_node_attr_matrix = torch.reshape(grad_node_attr_matrix, (max_node, num_features)) 
    grad_node_attr_matrix = (node_attr_matrix-baseline_node_attr_matrix) * grad_node_attr_matrix / steps
    grad_node_attr_matrix = grad_node_attr_matrix.numpy()

    return grad_node_attr_matrix   
        

5.2 calculate Integrated Gradient for a microstructure and save it in txt

In [ ]:
from numpy import savetxt
 
id=101
grad_node_attr_matrix=Intergrated_gradient_calculation(id, max_node, num_features, model)
savetxt("feature_grad_{0}.csv".format(id), grad_node_attr_matrix, delimiter=',')